# Suivi de trajectoire pour drone à voilure fixe

In [308]:
import os
import pygame
from pygame.math import Vector2
from math import *
import numpy as np
import random as rd

## Constants

### Global constants

In [309]:
k_position = 1
k_velocity = 1
k_angle_penalty = 1
k_velocity_penalty = 1
k_theta = 1
k_omega = 1

width = 1280
height = 720

### Swarm constants

In [310]:
N_drones = 5
launch_position = np.zeros(2)
k_collision_angle = 0.0001
k_collision_acceleration = 0.001

In [311]:
def signed_angle(u, v):
    angle = atan2(v[1], v[0]) - atan2(u[1], u[0])
    angle = degrees(angle)
    if angle > 180:
        angle -= 360
    elif angle < -180:
        angle += 360
    return angle

def add_gaussian_noise(vec, mean=0, std_dev=1, size = 2):
    noise = np.random.normal(mean, std_dev, size)
    return vec + noise

In [312]:
'''
class Vehicle:
    def __init__(self, x, y, scaling=1, angle_0=0.0, length_0=4, max_steering_0=30, max_acceleration_0=5.0, k_steering_0=30, brake_deceleration_0=10, free_deceleration_0=2, max_velocity_0=20):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle_0
        self.length = length_0
        self.max_acceleration = max_acceleration_0
        self.max_steering = max_steering_0
        self.max_velocity = max_velocity_0/scaling
        self.brake_deceleration = brake_deceleration_0
        self.free_deceleration = free_deceleration_0
        self.k_steering = k_steering_0
        self.scaling = scaling

        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt):
        # User input
        pressed = pygame.key.get_pressed()

        if pressed[pygame.K_UP]:
            if self.velocity.x < 0:
                self.acceleration = self.brake_deceleration
            else:
                self.acceleration += (1/self.scaling) * dt
        elif pressed[pygame.K_DOWN]:
            if self.velocity.x > 0:
                self.acceleration = -self.brake_deceleration
            else:
                self.acceleration -= (1/self.scaling) * dt
        elif pressed[pygame.K_SPACE]:
            if abs(self.velocity.x) > dt * self.brake_deceleration:
                self.acceleration = -copysign(self.brake_deceleration, self.velocity.x)
            else:
                self.acceleration = -self.velocity.x / dt
        else:
            if abs(self.velocity.x) > dt * self.free_deceleration:
                self.acceleration = -copysign(self.free_deceleration, self.velocity.x)
            else:
                if dt != 0:
                    self.acceleration = -self.velocity.x / dt
        self.acceleration = max(-self.max_acceleration, min(self.acceleration, self.max_acceleration))

        #steering
        if pressed[pygame.K_RIGHT]:
            self.steering -= self.k_steering*dt
        elif pressed[pygame.K_LEFT]:
            self.steering += self.k_steering*dt
        else:
            self.steering = 0
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))

        #velocity update
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(-self.max_velocity, min(self.velocity.x, self.max_velocity))


        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt
    
    def draw(self, car_image, ppu, game):
        rotated_car = pygame.transform.rotate(car_image, self.angle)
        rect = rotated_car.get_rect()
        game.screen.blit(rotated_car, self.position * ppu - (rect.width / 2, rect.height / 2))
        return
'''

'\nclass Vehicle:\n    def __init__(self, x, y, scaling=1, angle_0=0.0, length_0=4, max_steering_0=30, max_acceleration_0=5.0, k_steering_0=30, brake_deceleration_0=10, free_deceleration_0=2, max_velocity_0=20):\n        self.position = Vector2(x, y)\n        self.velocity = Vector2(0.0, 0.0)\n        self.angle = angle_0\n        self.length = length_0\n        self.max_acceleration = max_acceleration_0\n        self.max_steering = max_steering_0\n        self.max_velocity = max_velocity_0/scaling\n        self.brake_deceleration = brake_deceleration_0\n        self.free_deceleration = free_deceleration_0\n        self.k_steering = k_steering_0\n        self.scaling = scaling\n\n        self.acceleration = 0.0\n        self.steering = 0.0\n\n    def update(self, dt):\n        # User input\n        pressed = pygame.key.get_pressed()\n\n        if pressed[pygame.K_UP]:\n            if self.velocity.x < 0:\n                self.acceleration = self.brake_deceleration\n            else:\n 

In [313]:
class Vehicle:
    def __init__(self, x, y, angle=0.0, length=4, max_steering=40, max_acceleration=5.0, scaling=1):
        self.position = Vector2(10, 10)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle
        self.length = length
        self.max_acceleration = max_acceleration
        self.max_steering = max_steering
        self.max_velocity = 2
        self.brake_deceleration = 1
        self.free_deceleration = 0.2

        self.steering_acc = 0.0
        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt):
        self.acceleration = rd.gauss(1,2)
        self.acceleration = max(-self.max_acceleration, min(self.acceleration, self.max_acceleration))

        self.steering_acc = rd.gauss(0,70)
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(-self.max_velocity, min(self.velocity.x, self.max_velocity))
        self.steering += self.steering_acc*dt
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))
        if abs(self.steering) == self.max_steering:
            self.steering = 0
        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt

# Single drone pursuit

In [315]:
class Drone:
    def __init__(self, x, y, scaling=1, angle_0=0, length_0=2, max_steering_0=30, max_acceleration_0=5.0, k_steering_0=30,
                 brake_deceleration_0=10, free_deceleration_0=2, max_velocity_0=20, min_velocity_0=0,
                 target_position_0=Vector2(0,0), swarm_index_0=0):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle_0
        self.length = length_0
        self.max_acceleration = max_acceleration_0
        self.max_steering = max_steering_0
        self.max_velocity = max_velocity_0/scaling
        self.min_velocity = min_velocity_0/scaling
        self.brake_deceleration = brake_deceleration_0
        self.free_deceleration = free_deceleration_0
        self.k_steering = k_steering_0
        self.swarm_index = swarm_index_0
        try:
            self.previous_theta = acos(Vector2.dot(target_position_0, Vector2(1.0, 0.0))/Vector2.magnitude(target_position_0)) - self.angle
        except:
            self.previous_theta = 0
        self.acceleration = 0.0
        self.steering = 0.0

    def angle_calculation(self, target_position):
        v = Vector2.normalize(target_position - self.position) # à bruiter parce qu'on connait pas la position exacte
        #v = add_gaussian_noise(v,mean=0,std_dev=0.5)
        v[1] = -v[1] # attention ca donne l'angle dans la base "écran" sinon
        u = Vector2(1.0, 0.0).rotate(self.angle) #obtenable avec un gyroscope
        
        theta = signed_angle(u, v)
        return theta

    def update(self, dt, target_position, target_velocity, drone_positions=[]):
        # Deltas
        del_position = Vector2.magnitude(target_position - self.position)
        del_velocity = Vector2.magnitude(target_velocity - self.velocity)

        theta = self.angle_calculation(target_position)
        print(theta)
        #print(target_position.y, target_position.x)
        del_theta = (theta - self.previous_theta)

        self.previous_theta = theta

        collision_angle = 0
        collision_acceleration = 0

        # Avoiding collisions
        if (len(drone_positions) != 0):
            for i in range(len(drone_positions)):
                if (i != self.swarm_index):
                    # Angle
                    del_position = Vector2.magnitude(drone_positions[i] - self.position)
                    angle = self.angle_calculation(drone_positions[i])

                    if (self.position != drone_positions[i]):
                        angle = self.angle_calculation(drone_positions[i])
                        if ((angle >= -90) & (angle <= 90)):
                            collision_angle += (1.0/del_position)*cos(angle*pi/180.0)
                        else:
                            collision_angle -= (1.0/del_position)*cos(angle*pi/180.0)

                    # Acceleration
                    if ((angle >= 0) & (angle <= 180)):
                        collision_acceleration -= (1.0/del_position)*sin(angle*pi/180.0)
                    else:
                        collision_acceleration += (1.0/del_position)*sin(angle*pi/180.0)

        #Gaz pedal controller
        self.acceleration = k_position*del_position + k_velocity*del_velocity + k_collision_acceleration*collision_acceleration
        #self.acceleration = 0

        #steering controller
        self.steering = k_theta*theta + k_omega*del_theta + k_collision_angle*collision_angle - k_velocity_penalty*Vector2.magnitude(self.velocity)/self.max_velocity
        self.steering = k_theta*theta
        self.steering = max(-self.max_steering, min(self.steering, self.max_steering))

        #velocity update
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(self.min_velocity, min(self.velocity.x, self.max_velocity))


        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt

In [316]:
def draw(object, image, ppu, game):
    rotated_drone = pygame.transform.rotate(image, object.angle)
    rect = rotated_drone.get_rect()
    position = object.position * ppu - (rect.width / 2, rect.height / 2)
    position = [position[0]%width,position[1]%height]
    game.screen.blit(rotated_drone, position)
    return

In [317]:
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Drone pursuit")
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        car_image_path = "car.png"
        drone_image_path = "drone.png"
        car_image = pygame.image.load(car_image_path)
        drone_image = pygame.image.load(drone_image_path)
        taille_car = car_image.get_size()
        scaling_car = 10
        taille_car = [taille_car[0]/scaling_car, taille_car[1]/scaling_car]
        car_image = pygame.transform.scale(car_image, taille_car)

        taille_drone = drone_image.get_size()
        scaling_drone = 10
        taille_drone = [taille_drone[0]/scaling_drone, taille_drone[1]/scaling_drone]

        car_image = pygame.transform.scale(car_image, taille_car)
        drone_image = pygame.transform.scale(drone_image, taille_drone)

        car = Vehicle(x=10, y=10, scaling=scaling_car)
        drone = Drone(x=5, y=20, scaling=scaling_drone)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # Logic
            car.update(dt)

            drone.update(dt, target_position=car.position, target_velocity=car.velocity)
            
            # Drawing
            self.screen.fill((0, 0, 0))
            draw(car, car_image, ppu, self)
            draw(drone, drone_image, ppu, self)
            
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()

## Drone swarm pursuit

### Swarm class

In [318]:
class Swarm():
    def __init__(self, N_drones_0, launch_position_0):
        self.swarm_list = []
        self.drone_positions = []
        self.N_drones = N_drones_0
        for i in range(self.N_drones):
            position = launch_position_0 + np.random.normal(loc=0, scale=100, size=2)
            x = position[0]
            y = position[1]
            self.swarm_list.append(Drone(x=x, y=y, swarm_index_0=i))
            self.drone_positions.append(self.swarm_list[i].position)

    def update(self, dt, target_position, target_velocity):
        for i in range(self.N_drones):
            self.swarm_list[i].update(dt, target_position, target_velocity, self.drone_positions)
            self.drone_positions[i] = self.swarm_list[i].position

    def draw(self, drone_image, ppu, game):
        for i in range(self.N_drones):
            draw(self.swarm_list[i], drone_image, ppu, game)

### Swarm game

In [319]:
class Swarm_game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Drone pursuit")
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        car_image_path = "car.png"
        drone_image_path = "drone.png"
        car_image = pygame.image.load(car_image_path)
        drone_image = pygame.image.load(drone_image_path)
        taille_car = car_image.get_size()
        scaling_car = 10
        taille_car = [taille_car[0]/scaling_car, taille_car[1]/scaling_car]
        car_image = pygame.transform.scale(car_image, taille_car)

        taille_drone = drone_image.get_size()
        scaling_drone = 10
        taille_drone = [taille_drone[0]/scaling_drone, taille_drone[1]/scaling_drone]

        car_image = pygame.transform.scale(car_image, taille_car)
        drone_image = pygame.transform.scale(drone_image, taille_drone)

        car = Vehicle(x=10, y=10, scaling=scaling_car)
        swarm = Swarm(N_drones_0=N_drones, launch_position_0=launch_position)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # Logic
            car.update(dt)

            swarm.update(dt, target_position=car.position, target_velocity=car.velocity)
            
            # Drawing
            self.screen.fill((0, 0, 0))
            draw(car, car_image, ppu, self)
            swarm.draw(drone_image, ppu, self)
            
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()

## Run

### Swarm simulation

In [320]:
game = Swarm_game()
game.run()

-25.84229397757656
-155.4474368559914
-76.84011246165483
83.22405094943817
-38.01463988161697
-25.842249262953935
-155.44618592496684
-76.83995659107845
83.22359368132342
-38.01435697286539
-24.604083462474243
-155.05351669219633
-75.96395111135115
82.54326851141579
-37.32333010806692
-22.426163262506773
-154.3318900566759
-74.35072623548511
81.29052662932021
-36.05039820436955
-19.187217310848546
-153.1653811171115
-71.73216513838159
79.25748758539355
-33.98309056966145
-16.187796814407932
-151.7116843979949
-68.4524455342563
76.71109479486027
-31.3912301037287
-13.484482967048587
-149.76729418401032
-64.03262906379382
73.28020370742694
-27.894448412946435
-11.223328439566135
-147.4235570177695
-59.22380974668846
69.10961344541282
-23.916090903576425
-9.335971599362306
-144.68475442935667
-54.41238126485348
64.4235242448351
-20.047307836766
-7.855322418968463
-141.75940224602618
-49.880746271546506
60.00646777459376
-16.967864535775213
-6.529873011126978
-138.2602630181358
-45.0625507

KeyboardInterrupt: 

### Single drone simulation

In [ ]:
game = Game()
game.run()

63.43494882292201
63.43275648279115
63.4010826283388
63.33884113325751
63.244801606397225
63.11344202729705
62.93485673195996
62.73456530736029
62.5156769113402
62.262529577541144
61.97709605424981
61.637618460258146
61.29190137238248
60.94540401171264
60.614748910735045
60.26469241930477
59.91349544936923
59.56240893352839
59.20816820408459
58.85393120371495
58.49847041716215
58.14096695727597
57.79668612000396
57.44853438026501
57.07417026444756
56.69636445957125
56.32123330879961
55.94623169595771
55.57072476329438
55.19256514164902
54.81242272803537
54.442095847840044
54.04417952097103
53.64472870672595
53.25956364798283
52.85780097883138
52.47320036092031
52.0820177377939
51.6697915171941
51.276377037237545
50.86329554412641
50.455283209000136
50.06684033015087
49.674149132972566
49.27731135278069
48.87866016396409
48.461160536720314
48.062107503857774
47.661528745013484
47.25352501643448
46.823900439067714
46.39709504869643
45.985352871445194
45.547312761366385
45.110793940370506

KeyboardInterrupt: 